# [Lstm seq2seq - Keras Documentation](https://keras.io/examples/lstm_seq2seq/)
~Sequence to sequence example in Keras <b>(character-level).</b>~

<div style="border: solid 1px; padding: 0.5em 1em;">
<h3>Note:</h3>
It is not usefull to do <b>character-level</b> machine translation, as <b>word-level</b> models are more common in this domain.
</div>

## Overview
***

## Data Download
English to French sentence pairs are available at [Tab-delimited Bilingual Sentence Pairs](http://www.manythings.org/anki/), and set it to `./data/`

```sh
# データをダウンロードする。
$ wget http://www.manythings.org/anki/fra-eng.zip

# 解凍する。
$ unzip fra-eng.zip

# 指定の場所に移動する。
$ mv fra.txt ./data/

# いらないデータを削除する。
$ rm -rf _about.txt fra-eng.zip
```

In [1]:
# ファイルの中身を確認する。
! sed -n 1,5p ./data/fra.txt

Go.	Va !
Hi.	Salut !
Run!	Cours !
Run!	Courez !
Who?	Qui ?


In [2]:
# ファイルの行数を調べる。
! wc -l ./data/fra.txt

  167130 ./data/fra.txt


### Vectorize the data.
単語をベクトル化するために、語彙のデータを取得する。なお、
- \t を `<sos>`
- \n を `<eos>`

として利用する。（今回は end of character と言うべきかもしれないが笑）

In [3]:
import numpy as np

In [4]:
num_samples = 10000  # 使用する例文の数。
data_path = './data/fra.txt'

In [5]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: num_samples]:
    # 「英語 tab フラ語」という構成なので、そこで分ける。
    input_text, target_text = line.split('\t')
    # 「\t」が <sos>, 「\n」が <eos>の役割（今回は end of character だが笑） 
    target_text = '\t' + target_text + '\n'
    
    # それぞれの文章をリストに格納する。
    input_texts.append(input_text);
    target_texts.append(target_text)
    
    # 使われている文字を記録する。
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)

    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [6]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [7]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens :', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs :', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens : 70
Number of unique output tokens: 93
Max sequence length for inputs : 16
Max sequence length for outputs: 59


In [8]:
# 各文字に番号を振っていく。
input_token_index  = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

In [9]:
# データを格納する箱を用意しておく。
# エンコーダーで使われる (データ数, 文章の最大長, 文字の種類)
encoder_input_data  = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')
# デコーダーで使われる (データ数, 文章の最大長, 文字の種類)
decoder_input_data  = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

In [10]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        # i 番目のデータの t 文字目が index 表記だと input_token_index[char] なので、そこを 1 にする。
        encoder_input_data[i, t, input_token_index[char]] = 1.
        
    for t, char in enumerate(target_text):
        # 同様の処理をデコーダー側でも行う。
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # デコーダ側の正解データは入力文字の一つ後の文字なので、一文字ずらす。
            decoder_target_data[i, t-1, target_token_index[char]] = 1.

***

## Model

In [11]:
import keras

Using TensorFlow backend.


In [12]:
# パラメータの指定。
batch_size = 64
epochs = 100
latent_dim = 256  # 隠れ層(e)の次元。単語をベクトル化した時の特徴空間の次元数。

### Encoder

<img src="./img/vanilla-encoder.svg" width=40%>

In [13]:
encoder_inputs = keras.layers.Input(shape=(None, num_encoder_tokens), name="encoder-input")
# "return_state=True" にして、毎時刻の出力を得る。
encoder = keras.layers.LSTM(units=latent_dim, return_state=True, name="encoder-lstm")
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# `encoder_outputs` を無視し、それ以外を取得する。
encoder_states = [state_h, state_c]

Instructions for updating:
Colocations handled automatically by placer.


### Decoder

<img src="./img/vanilla-decoder.svg" width=40%>

In [14]:
# `encoder_states` を初期状態としたデコーダを作成。
decoder_inputs = keras.layers.Input(shape=(None, num_decoder_tokens), name="decoder-input")

# 訓練時にはデコーダーの出力を次の入力に使用せず、正解データを入力に使う。
# これを行わないと学習が完了しない、遅い、などの問題がある。
# もちろん推論時には一つ前のデコーダーの出力を入力として使う。
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True, name="decoder-lstm")
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation='softmax', name="output")
decoder_outputs = decoder_dense(decoder_outputs)

### Full-Model
`encoder_input_data` と `decoder_input_data` を受け取って `decoder_target_data` を出力するモデルを作成する。

In [15]:
model = keras.models.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [16]:
from keras.utils.vis_utils import plot_model
plot_model(model, "./img/training-model.png")

<img src="./img/training-model.png" width=30%>

### Training

In [17]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

```python
model.fit([encoder_input_data, decoder_input_data], 
          decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
```

```python
# Save model
model.save('./weights/s2s.h5')
```

In [18]:
model.load_weights('./weights/s2s.h5')

### Inference
学習時と推論時のモデルは少し異なるので、定義する。流れとしては、
1. 入力文字列をエンコードし、デコード時の初期状態(e)を取得する。
2. デコード時の初期状態 `<sos>` と e を入力として受け取り出力する。
3. 一つ前のデコーダーの出力とその時点のターゲット文字を入力として受け取り出力する、を繰り返す。

In [19]:
# encoder
encoder_model = keras.models.Model(encoder_inputs, encoder_states)

In [20]:
from keras.utils.vis_utils import plot_model
plot_model(encoder_model, "./img/inference-encoder-model.png")

<img src="./img/inference-encoder-model.png" width=20%>

In [21]:
# decoder
decoder_state_input_h = keras.layers.Input(shape=(latent_dim,), name="hidden-input")
decoder_state_input_c = keras.layers.Input(shape=(latent_dim,), name="source-input")
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

In [22]:
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

In [23]:
decoder_model = keras.models.Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

In [24]:
from keras.utils.vis_utils import plot_model
plot_model(decoder_model, "./img/inference-decoder-model.png")

<img src="./img/inference-decoder-model.png" width=50%>

***
|Training|Inference|
|:--:|:--:|
|<img src="./img/training-model.png">|<img src="./img/inference-encoder-model.png"><img src="./img/inference-decoder-model.png">|
***

In [25]:
# 逆引き用の辞書。インデックスからcharacterを出力する。
reverse_input_char_index  = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [26]:
# ビームサーチを行わず、各ステップで最も予測確率が高いものを選択する。
def decode_sequence(input_seq):
    # デコード時の隠れ層の初期状態を求める。 [(1, 256), (1, 256)]
    states_value = encoder_model.predict(input_seq)

    # デコーダーの入力（ターゲットトークン）は、同じものを使い続ける。
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # <sos> を定義する。
    target_seq[0, 0, target_token_index['\t']] = 1.

    decoded_sentence = '' # デコードされた文字をつなげていく。
    while True:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 最も予測確率が高いものを出力とする。
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # <eos>を出力するか、最大長を超えたら終了する。
        if (sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_length):
            break

        # 前のデコーダーの出力結果を入力にする。
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # 状態も更新する。
        states_value = [h, c]

    return decoded_sentence

In [27]:
# 確認(とはいえこれは訓練データ)
for seq_index in range(5):
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-'*20)
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

--------------------
Input sentence: Go.
Decoded sentence: Va !

--------------------
Input sentence: Hi.
Decoded sentence: Salut !

--------------------
Input sentence: Run!
Decoded sentence: Courez !

--------------------
Input sentence: Run!
Decoded sentence: Courez !

--------------------
Input sentence: Who?
Decoded sentence: Qui ?

